# Insanely-Fast-Whisper Notebook

This repository contains a Jupyter Notebook interface for **speech-to-text (STT)** based on the open-source project **insanely-fast-whisper**:  
https://github.com/Vaibhavs10/insanely-fast-whisper

The notebook wraps the original command-line tool and enables **high-speed transcription and speaker diarization** of audio recordings, with a particular focus on **Danish interview data**. All processing is performed in a **private cloud environment**, ensuring that sensitive audio data remains within controlled infrastructure and that privacy requirements are preserved.

In addition to transcription and diarization, the notebook includes functionality to **export transcripts directly to Microsoft Word (.docx)** format, making the output suitable for analysis, reporting, and archival purposes.

## GPU and Memory Requirements

With the default configuration, *insanely-fast-whisper* is **GPU-intensive** and requires a **significant amount of GPU memory (VRAM)** to achieve high transcription speed. Systems with limited VRAM may encounter out-of-memory errors when using the default settings.

If GPU memory is constrained, the **batch size can be reduced** to lower VRAM consumption, at the cost of reduced throughput. Adjusting the batch size allows the notebook to run on a wider range of GPU hardware while maintaining functional correctness.

## Maintenance and Dependency Management

The original *insanely-fast-whisper* project is no longer actively maintained, and its dependencies were not pinned to fixed versions. As a result, updates to upstream libraries have frequently caused the project to stop working.

To ensure long-term stability, **all dependencies were pinned to known working versions in late 2025**, allowing the tool to continue functioning reliably despite changes in the surrounding software ecosystem.

A maintained fork with pinned dependencies is available here:  
https://github.com/putssander/insanely-fast-whisper

This notebook is designed to work with that forked version and should be used in conjunction with it to ensure reproducible and stable results.


# Quickrun for ease of use

### Configuration

To use this notebook, you need to set up your Hugging Face token.

1.  Create a file named `.env` in the root directory of this project (same level as `notebooks/` folder).
2.  Add your Hugging Face token to the `.env` file like this:
    ```
    HUGGINGFACE_AUTH_TOKEN=your_token_here
    ```
3.  Alternatively, if you don't create a `.env` file, the notebook will ask you to enter your token interactively when needed.


In [ ]:
import os, glob, re

# Load environment variables from .env file if it exists
if os.path.exists('.env'):
    with open('.env', 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

# set these 3 variables and run all!
# HUGGINGFACE_AUTH_TOKEN is loaded from .env or will be requested interactively later
extensions = ["mp3", "WMA", "wma", "m4a", "wav"]  # List of extensions
path_audio = '/work/speech/Kasper/Prætest/'

# path_audio = '/work/speech/Test/single/'

# Use glob to match multiple extensions
files = []
for ext in extensions:
    path_audio_regex = f"{path_audio}*.{ext}"  # glob does not use regex
    files.extend(glob.glob(path_audio_regex))

length = len(files)
print(f"The following {length} files will be transcribed: \n{files}")


The following 2 files will be transcribed: 
['/work/speech/Kasper/Prætest/P4.WMA', '/work/speech/Kasper/Prætest/P9.WMA']


### 1. Install

In [ ]:
!pip install -U pip pipx
!pipx ensurepath

# Remove any existing installation
!pipx uninstall insanely-fast-whisper || true

# Install pinned fork directly from GitHub
!pipx install git+https://github.com/putssander/insanely-fast-whisper.git --force

# Dependency for Word export
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pipx]
Nothing to uninstall for insanely-fast-whisper 😴
  installed package insanely-fast-whisper 0.0.16, installed using Python 3.12.12
  These apps are now globally available
    - insanely-fast-whisper
done! ✨ 🌟 ✨
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 197.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-docx] [python-docx]


#### 1.B HuggingFace user agreements!
To run this notebook without the following is required.
- a huggingface account [link](https://huggingface.co/)
- a huggingface authentication token [link](https://huggingface.co/settings/tokens)
- accept 2 user agreements:
  - [https://huggingface.co/pyannote/speaker-diarization-3.1](https://huggingface.co/pyannote/speaker-diarization-3.1)
  - [https://hf.co/pyannote/segmentation-3.0](https://hf.co/pyannote/segmentation-3.0)


In [4]:
# Run this cell, copy paste your huggingface token and press enter
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

if not os.environ.get("HUGGINGFACE_AUTH_TOKEN"):
    _set_env("HUGGINGFACE_AUTH_TOKEN")

### 2. Transcribe

In [5]:
import re
import os

LANGUAGE = "da"
NUM_SPEAKERS = 2
TASK = "transcribe"
DIARIZATION_MODEL = "pyannote/speaker-diarization-3.1"
HUGGINGFACE_AUTH_TOKEN = os.environ['HUGGINGFACE_AUTH_TOKEN']
BATCH_SIZE = 16


def transcribe(file_name):
    TRANSCRIPT_PATH = os.path.splitext(file_name)[0]+"-transcription.json"
    FILE_NAME = re.escape(file_name)
    TRANSCRIPT_PATH = re.escape(TRANSCRIPT_PATH)    
    
    # workaround, current MPLBACKEND default does not work
    os.environ['MPLBACKEND'] = 'svg'
    # !echo $MPLBACKEND
    !insanely-fast-whisper --file-name {FILE_NAME} --transcript-path {TRANSCRIPT_PATH} --language {LANGUAGE} --task {TASK} --diarization_model {DIARIZATION_MODEL} --hf-token {HUGGINGFACE_AUTH_TOKEN} --batch-size {BATCH_SIZE}


In [6]:
# Ensure the kernel process can find pipx-installed CLIs for this session
import os, pathlib
pipx_bin = str(pathlib.Path.home() / '.local' / 'bin')
os.environ['PATH'] = pipx_bin + ':' + os.environ.get('PATH', '')
print('Prepended to PATH for current kernel session:', os.environ['PATH'])
# After running this cell, `!insanely-fast-whisper` should be available in subsequent cells

Prepended to PATH for current kernel session: /home/ucloud/.local/bin:/home/ucloud/.cargo/bin:/opt/go/Projects/Proj1/bin:/usr/local/go/bin:/opt/conda/bin:/home/ucloud/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin:/home/ucloud/.local/share/coursier/bin


In [7]:
for idx, file in enumerate(files):
    print(f"file {idx+1} of {length} file={file}")
    transcribe(file)

file 1 of 2 file=/work/speech/Kasper/Prætest/P4.WMA
config.json: 1.27kB [00:00, 8.58MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
model.safetensors: 100%|████████████████████| 3.09G/3.09G [00:03<00:00, 999MB/s]
generation_config.json: 3.90kB [00:00, 33.4MB/s]
tokenizer_config.json: 283kB [00:00, 600MB/s]
vocab.json: 1.04MB [00:00, 95.4MB/s]
tokenizer.json: 2.48MB [00:00, 317MB/s]
merges.txt: 494kB [00:00, 190MB/s]
normalizer.json: 52.7kB [00:00, 179MB/s]
added_tokens.json: 34.6kB [00:00, 172MB/s]
special_tokens_map.json: 2.07kB [00:00, 21.5MB/s]
preprocessor_config.json: 100%|████████████████| 340/340 [00:00<00:00, 6.79MB/s]
Device set to use cuda:0
🤗 Transcribing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:00Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warn

### 3. Convert transcriptions to Word

In [ ]:
import glob
import os
import json
import re

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH


def diarization_to_docx_edit(audio_file_diarization):

    audio_file_basename = os.path.basename(audio_file_diarization)
    
    doc = Document()
    
    paragraph = doc.add_paragraph()
    paragraph.add_run(audio_file_basename).bold = True
    
    paragraph_format = paragraph.paragraph_format
    paragraph_format.alignment
    paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    f = open(audio_file_diarization, 'r', encoding="utf-8")

    data = json.load(f)

    for entry in data["speakers"]:
        paragraph = doc.add_paragraph()
        paragraph.add_run(str(entry["timestamp"]) + "\n")
        paragraph.add_run(str(entry["speaker"])  + "\n")
        paragraph.add_run(str(entry["text"]).strip())

            
    docx_filename = audio_file_diarization.replace(".json", "_edit.docx")
    doc.save(docx_filename)


# absolute path containing the json diarization files
extension = "json"
path_json_regex = r'' + re.escape(path_audio) + '*.' + re.escape(extension)
files_json = glob.glob(path_json_regex)
length_json = len(files_json)

print(f"The following {length_json} files will be converted to docx {files_json}")

for idx, file in enumerate(files_json):
    print(f"file {idx+1} of {length} file={file}")
    diarization_to_docx_edit(file)
print(f"Done!")


The following 2 files will be converted to docx ['/work/speech/Kasper/Prætest/P9-transcription.json', '/work/speech/Kasper/Prætest/P4-transcription.json']
file 1 of 2 file=/work/speech/Kasper/Prætest/P9-transcription.json
file 2 of 2 file=/work/speech/Kasper/Prætest/P4-transcription.json
Done!
